# 📊 מדריך מקיף לניתוח סדרות זמן עם Polars

## פרק 9: Time Series Analysis + Forecasting

---

### 👋 ברוכים הבאים!

מדריך זה יעזור לכם ללמוד כיצד לעבוד עם סדרות זמן (Time Series) באמצעות ספריית **Polars**.

### 🎯 מה נלמד במדריך זה?

1. **עבודה עם תאריכים ושעות** - המרה, פירוק, וסינון
2. **חלונות מתגלגלים (Rolling Windows)** - חישוב ממוצעים נעים
3. **דגימת יתר ותת-דגימה (Resampling)** - שינוי רזולוציית הזמן
4. **חיזוי סדרות זמן (Forecasting)** - חיזוי עם Functime 🔮

### 📚 תוכן עניינים

- [1. הכנה - טעינת נתונים](#section-1)
- [2. עבודה עם תאריכים ושעות](#section-2)
- [3. חלונות מתגלגלים (Rolling Windows)](#section-3)
- [4. טכניקות דגימה מחדש (Resampling)](#section-4)
- [5. חיזוי סדרות זמן (Forecasting)](#section-5)
- [6. תרגילים אינטראקטיביים](#section-6)
- [7. סיכום וטיפים](#section-7)

---

<a id='section-1'></a>
## 1. 🚀 הכנה - טעינת נתונים

נתחיל בטעינת קובץ CSV שמכיל נתוני מזג אוויר מטורונטו.

In [ ]:
# ייבוא הספרייה
%pip install -U hvplot
%pip install "scipy<1.8.0"
import polars as pl

# טעינת הנתונים כ-LazyFrame
lf = pl.scan_csv('../data/toronto_weather.csv')

# הצגת 5 השורות הראשונות
lf.head().collect()

### 🌡️ המרת טמפרטורה מקלווין לצלזיוס

In [ ]:
# המרה לצלזיוס
lf = lf.with_columns(
    (pl.col('temperature') - 273.15).round(2).alias('temperature')
)

# בדיקת התוצאה
lf.head().collect()

---
<a id='section-2'></a>
## 2. 📅 עבודה עם תאריכים ושעות

### ניתוח תאריכים אוטומטי

In [ ]:
# טעינה עם ניתוח תאריכים אוטומטי
lf_date_parsed = pl.scan_csv('../data/toronto_weather.csv', try_parse_dates=True)

# הצגת הנתונים
lf_date_parsed.head().collect()

### המרת תאריכים ידנית

In [ ]:
# המרה ידנית של עמודת datetime
lf = lf.with_columns(
    pl.col('datetime').str.to_datetime()
)

lf.head().collect()

### פירוק תאריכים לרכיבים

In [ ]:
# פירוק התאריך לרכיבים
(
    lf
    .select(
        'datetime',
        pl.col('datetime').dt.year().alias('year'),
        pl.col('datetime').dt.month().alias('month'),
        pl.col('datetime').dt.day().alias('day'),
        pl.col('datetime').dt.time().alias('time')
    )
    .head().collect()
)

### סינון לפי תאריכים

In [ ]:
from datetime import datetime

# סינון לשנת 2017, רק בשעות הבוקר (עד 12:00)
filtered_lf = (
    lf
    .filter(
        pl.col('datetime').dt.date().is_between(
            datetime(2017, 1, 1),
            datetime(2017, 12, 31)
        ),
        pl.col('datetime').dt.hour() < 12
    )
)

filtered_lf.head().collect()

---
<a id='section-3'></a>
## 3. 📈 חלונות מתגלגלים (Rolling Windows)

### ממוצע נע פשוט

In [ ]:
# ממוצע נע של 3 שעות על הטמפרטורה
(
    lf
    .select(
        'datetime',
        'temperature',
        pl.col('temperature').rolling_mean(3).alias('3hr_rolling_avg')
    )
    .head()
    .collect()
)

### ממוצעים יומיים

In [ ]:
# חישוב ממוצע טמפרטורה יומי
daily_avg_temperature_lf = (
    lf
    .select(
        pl.col('datetime').dt.date().alias('date'),
        'temperature'
    )
    .group_by('date', maintain_order=True)
    .agg(
        pl.col('temperature').mean().alias('daily_avg_temp')
    )
)

daily_avg_temperature_lf.head().collect()

### סטטיסטיקות מתגלגלות

In [ ]:
# חישוב ממוצע, מינימום ומקסימום נעים של 3 ימים
(
    daily_avg_temperature_lf
    .select(
        'date',
        'daily_avg_temp',
        pl.col('daily_avg_temp').rolling_mean(3).alias('3day_rolling_avg'),
        pl.col('daily_avg_temp').rolling_min(3).alias('3day_rolling_min'),
        pl.col('daily_avg_temp').rolling_max(3).alias('3day_rolling_max')
    )
    .head()
    .collect()
)

---
<a id='section-4'></a>
## 4. ⏱️ טכניקות דגימה מחדש (Resampling)

### תת-דגימה (Downsampling)

In [ ]:
# דגימה מחדש לשבועות
(
    lf
    .set_sorted('datetime')
    .group_by_dynamic(
        'datetime',
        every='1w'
    )
    .agg(
        pl.col('humidity').mean().round(1)
    )
    .head(10)
    .collect()
)

### דגימת יתר (Upsampling)

In [ ]:
# דגימת יתר - נתונים כל 30 דקות
upsampled_df = (
    lf
    .set_sorted('datetime')
    .collect()
    .upsample(
        time_column='datetime',
        every='30m',
        maintain_order=True
    )
    .select(
        'datetime',
        pl.col('humidity')
    )
)

upsampled_df.head(10)

### אינטרפולציה

In [ ]:
# מילוי הערכים החסרים עם אינטרפולציה
(
    upsampled_df
    .with_columns(
        pl.col('humidity').interpolate()
    )
    .head(10)
)

---
<a id='section-5'></a>
## 5. 🔮 חיזוי סדרות זמן עם Functime

### מהו Time Series Forecasting?

**חיזוי סדרות זמן** הוא תהליך של חיזוי ערכים עתידיים על בסיס נתונים היסטוריים.

### מה זה Functime?

**Functime** היא ספרייה מתקדמת לחיזוי סדרות זמן שפותחה במיוחד לעבוד עם **Polars**.

**יתרונות:**
- ⚡ **מהירות** - אופטימיזציה מלאה ל-Polars
- 🎯 **פשטות** - API נקי וקל לשימוש
- 🔧 **גמישות** - מגוון מודלים וטכניקות

### 🚀 הכנת הנתונים

In [ ]:
# טעינת נתוני טמפרטורות היסטוריות
lf_hist = pl.scan_csv('../data/historical_temperatures.csv', try_parse_dates=True)

# הצגת דוגמה
lf_hist.head().collect()

In [ ]:
# רשימת הערים
lf_hist.select('city').unique().sort('city').collect()

In [ ]:
# הצגת 3 שורות ראשונות לכל עיר
lf_hist.group_by('city').head(3).collect()

### 📊 צבירה חודשית

In [ ]:
# קבלת שמות העמודות
# time_col, entity_col, value_col = lf_hist.collect_schema().names()
cols = lf_hist.collect().columns
if len(cols) < 3:
    raise ValueError("expected at least 3 columns after .collect()")
time_col, entity_col, value_col = cols[:3]
print(f"עמודות: {time_col}, {entity_col}, {value_col}")

# צבירה חודשית + המרה לצלזיוס
y = (
    lf_hist
    .group_by_dynamic(
        time_col,
        every='1mo',
        group_by=entity_col,
    )
    .agg(
        (pl.col('temperature').mean() - 273.15).round(1),
    )
)

y.group_by('city').head(3).collect()

# python
# Monkeypatch scipy.signal.ricker if missing (run this before importing functime)
import numpy as np
import importlib

try:
    import scipy.signal as _signal
except Exception:
    raise

if not hasattr(_signal, "ricker"):
    def ricker(points, a):
        # lightweight Ricker (Mexican hat) approximation compatible with old scipy API:
        # inputs: points (int), a (float width)
        t = np.arange(points) - (points - 1.0) / 2.0
        xsq = (t / a) ** 2
        return (1 - xsq) * np.exp(-xsq / 2.0)
    _signal.ricker = ricker

# Now import functime (or call your create_train_test_sets)
from functime.cross_validation import train_test_split
# ... proceed to call create_train_test_sets(...) as before### 🎯 פיצול לנתוני אימון ובדיקה

In [ ]:
# python
# Monkeypatch scipy.signal.ricker if missing (run this before importing functime)
import numpy as np
import importlib

try:
    import scipy.signal as _signal
except Exception:
    raise

if not hasattr(_signal, "ricker"):
    def ricker(points, a):
        # lightweight Ricker (Mexican hat) approximation compatible with old scipy API:
        # inputs: points (int), a (float width)
        t = np.arange(points) - (points - 1.0) / 2.0
        xsq = (t / a) ** 2
        return (1 - xsq) * np.exp(-xsq / 2.0)
    _signal.ricker = ricker

# Now import functime (or call your create_train_test_sets)
from functime.cross_validation import train_test_split
# ... proceed to call create_train_test_sets(...) as before

In [ ]:
from functime.cross_validation import train_test_split

# פיצול
test_size = 3
X = y.select(entity_col, time_col)
y_train, y_test = (
    y
    .select(entity_col, time_col, value_col)
    .pipe(train_test_split(test_size))
)
X_train, X_test = X.pipe(train_test_split(test_size))

print(f"Train: {y_train.collect().shape[0]} שורות")
print(f"Test: {y_test.collect().shape[0]} שורות")

### 🤖 אימון מודל

In [ ]:
# python
# Patch `functime.conversion` to avoid Polars zero-copy restriction.
import numpy as np
import polars as pl
import functime.conversion as conv

def _df_to_ndarray(df: pl.DataFrame, n_groups=None) -> np.ndarray:
    cols = list(df.columns)[2:]
    n_rows = df.height
    n_cols = len(cols)
    X = np.empty((n_rows, n_cols), dtype=np.float32)
    for i, col in enumerate(cols):
        series = df.get_column(col)
        arr = series.to_numpy(zero_copy_only=False)
        X[:, i] = np.asarray(arr, dtype=np.float32)
    return X

def _X_to_numpy(X: pl.DataFrame) -> np.ndarray:
    df = (
        X.lazy()
        .select(pl.col(X.columns[2:]).cast(pl.Float32))
        .select(
            pl.when(pl.all().is_infinite() | pl.all().is_nan())
            .then(None)
            .otherwise(pl.all())
            .keep_name()
        )
        .fill_null(strategy="mean")
        .collect(streaming=True)
    )
    return _df_to_ndarray(df)

def _y_to_numpy(y: pl.DataFrame) -> np.ndarray:
    ser = (
        y.lazy()
        .select(pl.col(y.columns[-1]).cast(pl.Float32))
        .select(
            pl.when(pl.all().is_infinite() | pl.all().is_nan())
            .then(None)
            .otherwise(pl.all())
            .keep_name()
        )
        .fill_null(strategy="mean")
        .collect(streaming=True)
        .get_column(y.columns[-1])
    )
    arr = ser.to_numpy(zero_copy_only=False)
    return np.asarray(arr, dtype=np.float32)

# Apply patches
conv.df_to_ndarray = _df_to_ndarray
conv.X_to_numpy = _X_to_numpy
conv.y_to_numpy = _y_to_numpy

# Now safe to call forecasting code (run this before calling `forecaster.fit`)# Patch functime conversion to avoid Polars zero-copy restriction.
# Run this before importing/using functime.forecasting or calling forecaster.fit.

import numpy as np
import importlib

# Import the module to patch (module may already be loaded)
import functime.conversion as conv

def _df_to_ndarray(df, n_groups=None):
    # keep same signature as original; build a NumPy (float32) matrix
    cols = list(df.columns)[2:]
    n_rows = df.height
    n_cols = len(cols)
    X = np.empty((n_rows, n_cols), dtype=np.float32)
    for i, col in enumerate(cols):
        series = df.get_column(col)
        # allow copying if zero-copy isn't possible
        arr = series.to_numpy(zero_copy_only=False)
        X[:, i] = np.asarray(arr, dtype=np.float32)
    return X

# Apply patch
conv.df_to_ndarray = _df_to_ndarray

# Now import and use the forecaster as before
from functime.forecasting import linear_model

# יצירת המודל
forecaster = linear_model(lags=24, freq='1mo')
# forecaster.fit(y=y_train)  # safe to call now

# אימון
forecaster.fit(y=y_train)

# חיזוי
y_pred = forecaster.predict(fh=test_size)

print("חיזוי הושלם!")
y_pred.head()

### 📊 הערכת דיוק

In [ ]:
from functime.metrics import mase

# חישוב דיוק
scores = mase(
    y_true=y_test,
    y_pred=y_pred,
    y_train=y_train
)

print("ציוני MASE:")
print(scores)

### 📈 ויזואליזציה

In [ ]:
# גרף הנתונים האמיתיים
actual_viz = (
    y.collect()
    .plot.line(
        x='datetime',
        y='temperature',
        by='city',
        subplots=True
    )
    .cols(2)
)

# גרף החיזויים
pred_viz = (
    y_pred
    .plot.line(
        x='datetime',
        y='temperature',
        by='city',
        subplots=True
    )
    .cols(2)
)

# שילוב
actual_viz * pred_viz

### 🔧 Feature Engineering מתקדם

In [ ]:
from functime.seasonality import add_calendar_effects

# בנה את התכונות
y_features_base = (
    lf_hist
    .group_by_dynamic(
        time_col,
        every='1mo',
        group_by=entity_col,
    )
    .agg([
        (pl.col('temperature').mean() - 273.15).round(1).alias('temp_mean'),
        pl.col(value_col).mean().alias('value_mean'),
        pl.col(value_col).std().alias('value_std'),
    ])
)

# הוסף calendar effects - שים לב שזה מחזיר Transformer!
transformer = add_calendar_effects(['month'])

# החל את ה-Transformer
y_features = transformer(y_features_base)  # או transformer.transform(y_features_base)

# עכשיו אפשר להשתמש
y_features.head().collect()

### 📝 סיכום - חיזוי

**למדנו:**
- ✅ מהו חיזוי סדרות זמן
- ✅ הכנת נתונים לחיזוי
- ✅ פיצול כרונולוגי train/test
- ✅ אימון מודל Linear עם Lags
- ✅ הערכת דיוק עם MASE
- ✅ ויזואליזציה
- ✅ Feature Engineering

**💡 מודלים נוספים:**
```python
from functime.forecasting import (
    linear_model,
    lightgbm,  # מומלץ!
    xgboost,
    knn,
    catboost
)
```

---
<a id='section-6'></a>
## 6. 🎓 תרגילים אינטראקטיביים

### תרגיל 1: סינון ופירוק תאריכים ⭐

**משימה:** סנן לשנת 2015, הוסף עמודת יום בשבוע, וחשב ממוצע לכל יום.

In [ ]:
# הקוד שלך כאן:



### תרגיל 2: ממוצע נע ⭐⭐

**משימה:** חשב ממוצעים נעים של 7 ו-30 ימים וצור גרף.

In [ ]:
# הקוד שלך כאן:



### תרגיל 3: חיזוי עם LightGBM ⭐⭐⭐⭐

**משימה:** אמן מודל LightGBM במקום Linear וחזה 6 חודשים.

In [ ]:
# הקוד שלך כאן:
# from functime.forecasting import lightgbm



---
<a id='section-7'></a>
## 7. 📚 סיכום וטיפים

### 🎯 מה למדנו?

#### 1. עבודה עם תאריכים
- ✅ טעינה אוטומטית עם `try_parse_dates`
- ✅ פירוק לרכיבים
- ✅ סינון לפי טווחים

#### 2. חלונות מתגלגלים
- ✅ `rolling_mean`, `rolling_min`, `rolling_max`
- ✅ שיטות שונות
- ✅ ויזואליזציה

#### 3. Resampling
- ✅ Downsampling עם `group_by_dynamic`
- ✅ Upsampling עם `upsample`
- ✅ אינטרפולציה

#### 4. חיזוי 🔮
- ✅ אימון מודלים
- ✅ הערכת דיוק
- ✅ Feature Engineering

### 💡 10 טיפים חשובים

1. **תמיד בדוק סוגי נתונים**
2. **השתמש ב-set_sorted()** לפני פעולות זמן
3. **LazyFrame vs DataFrame** - דע מתי להשתמש
4. **בדוק עם head() קודם**
5. **prefer built-in functions**
6. **ויזואליזציה** - תמיד
7. **maintain_order=True** בgroup_by
8. **min_periods=1** למניעת nulls
9. **פצל כרונולוגית** בחיזוי
10. **תעד את הקוד** שלך

### 📖 משאבים נוספים

- **Polars:** https://pola-rs.github.io/polars/
- **Functime:** https://docs.functime.ai/
- **Discord:** https://discord.gg/4UfP5cfBE7

---

## 🎉 סיימתם!

כל הכבוד! עכשיו אתם יודעים לעבוד עם סדרות זמן ב-Polars **ואפילו לחזות את העתיד**! 🔮

**בהצלחה!** 🌟